In [158]:
import spacy
import pandas as pd
from __future__ import unicode_literals, print_function
import plac
from random import shuffle
from pathlib import Path
from spacy.util import minibatch, compounding
import os
import ast
import random 

In [92]:
n_iter = 128
base_df = pd.read_csv('../datasets/TrainNER.csv', delimiter=';', encoding='cp1252')
print('base_df')
base_df.describe()

base_df


,Sentence #,Word,POS,Tag
count,9000,196645,196645,196645
unique,9000,15323,41,15
top,Sentence: 1367,the,NN,O
freq,1,9839,27417,166610


In [108]:
for index, row in base_df.iterrows():
    if str(row['Sentence #']) != 'nan':
        current_sentence = int(row['Sentence #'].split(' ')[1])
    base_df.loc[index, 'SentenceID'] = current_sentence
base_df.loc[len(base_df.index), 'Sentence #'] = 9001

In [232]:
def get_sentence(id):
    words = base_df[(base_df['SentenceID'] == id)]['Word'].values.tolist()
    return ' '.join(str(word) for word in words)

def get_sentence_entities(sentence):
    entities = {'entities': None}
    entity = []
    for word in sentence.split(' '):
        entity.append((sentence.find(word), sentence.find(word)+len(word), str(base_df[base_df['Word'] == word]['Tag'].iloc[0])))
    entities['entities'] = entity
    return entities

def get_acc(df):
    tp = df.loc[df['Tag'] == df['Prediction']].shape[0]
    return tp / df.shape[0] * 100

def get_recall(df):
    tp = df.loc[(df['Tag'] != 'O') & (df['Tag'] == df['Prediction'])].shape[0]
    tp_fn = df.loc[df['Tag'] != 'O'].shape[0]
    return tp / tp_fn * 100

def get_precision(df):
    tp = df.loc[(df['Tag'] != 'O') & (df['Tag'] == df['Prediction'])].shape[0]
    tp_fp = df.loc[df['Prediction'] != 'O'].shape[0]
    return tp / tp_fp

In [234]:
# sentence_df = pd.DataFrame(columns=["Sentence", "Entities"])
# for i in base_df['SentenceID'].unique().tolist():
#     sentence = get_sentence(i)
#     entities = {'entities': None}
#     entity = []
#     try:
#         for word in sentence.split(' '):
#             entity.append((sentence.find(word), sentence.find(word)+len(word), str(base_df[base_df['Word'] == word]['Tag'].iloc[0])))
#         entities['entities'] = entity
#         sentence_df = sentence_df.append({'Sentence': sentence, 'Entities': entities}, ignore_index=True)
#     except:
#         print("unfurtunatelly an error has ocurred")

# sentence_df.to_csv('../datasets/SentenceEntities.csv')

In [142]:
# sentence_df.to_csv('../datasets/SentenceEntities.csv', sep=';')

In [166]:
rands = random.sample(range(9000), 500)
train_data = []
for num in rands:
    sent = get_sentence(num)
    entities = get_sentence_entities(sent)
    train_data.append((sent, entities))
train_data

[('Ankara signed an agreement with Baghdad last month that commits Iraqi troops to crack down on PKK rebels who Turkey says use northern Iraq to attack Turkey .',
  {'entities': [(0, 6, 'B-geo'),
    (7, 13, 'O'),
    (14, 16, 'O'),
    (17, 26, 'O'),
    (27, 31, 'O'),
    (32, 39, 'B-geo'),
    (40, 44, 'O'),
    (45, 50, 'O'),
    (51, 55, 'O'),
    (56, 63, 'O'),
    (64, 69, 'B-gpe'),
    (70, 76, 'O'),
    (77, 79, 'O'),
    (80, 85, 'O'),
    (86, 90, 'O'),
    (46, 48, 'O'),
    (94, 97, 'B-org'),
    (98, 104, 'O'),
    (105, 108, 'O'),
    (109, 115, 'B-geo'),
    (116, 120, 'O'),
    (121, 124, 'O'),
    (125, 133, 'O'),
    (64, 68, 'I-org'),
    (77, 79, 'O'),
    (142, 148, 'O'),
    (109, 115, 'B-geo'),
    (156, 157, 'O')]}),
 ('It also warns against working in places where Arabs are employed .',
  {'entities': [(0, 2, 'O'),
    (3, 7, 'O'),
    (8, 13, 'O'),
    (14, 21, 'O'),
    (22, 29, 'O'),
    (17, 19, 'O'),
    (33, 39, 'O'),
    (40, 45, 'O'),
    (46, 51, 'B-g

In [46]:
# train.loc['entities'] = train.apply(lambda row: {'entities':[(0, len(str(row['Word'])), str(row['Tag']))]}, axis=1)
# train['Word'] = train['Word'].apply(str)
# subset = train[['Word', 'entities']]
# train_tuples = [tuple(x) for x in subset.values]

/Users/krisztian/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/krisztian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/krisztian/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/in

In [167]:
nlp = spacy.blank('en')
ner = nlp.create_pipe('ner')
nlp.add_pipe(ner, last=True)

for _, annotations in train_data:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

# for index, row in train.iterrows():
#     ner.add_label(str(row['Tag']))
    
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        shuffle(train_data)
        losses = {}
        
        batches = minibatch(train_data, size=compounding(4., 32., 1.001))
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(
                texts,
                annotations,
                drop=0.5,
                sgd=optimizer,
                losses=losses
            )
        print(itn, losses)

0 {'ner': 466.8611622277815}
1 {'ner': 341.99028334773624}
2 {'ner': 280.15690776688916}
3 {'ner': 239.25381801651656}
4 {'ner': 226.08756555516447}
5 {'ner': 206.5330889001798}
6 {'ner': 189.86559832009698}
7 {'ner': 177.23115780317676}
8 {'ner': 168.00044872864237}
9 {'ner': 156.31934294613166}
10 {'ner': 148.4241630763487}
11 {'ner': 140.90928390705562}
12 {'ner': 131.32132554632483}
13 {'ner': 126.73481491346774}
14 {'ner': 113.96380567314787}
15 {'ner': 104.6551895963223}
16 {'ner': 100.70400207968174}
17 {'ner': 95.2911575078916}
18 {'ner': 90.12093669706206}
19 {'ner': 87.6971546631121}
20 {'ner': 81.41982908185207}
21 {'ner': 77.82846593070076}
22 {'ner': 70.97996267866105}
23 {'ner': 67.66389455920435}
24 {'ner': 67.36442953249914}
25 {'ner': 62.76715846041457}
26 {'ner': 58.98455456231359}
27 {'ner': 61.31372813404416}
28 {'ner': 55.856990167769524}
29 {'ner': 56.2641530673202}
30 {'ner': 55.66107885960159}
31 {'ner': 50.804594462743445}
32 {'ner': 49.987594137414405}
33 {'ne

In [228]:
predicted_df = base_df
sentenceID = 1
doc = nlp(get_sentence(sentenceID))
for index, row in predicted_df.iterrows():
    if sentenceID != row['SentenceID']:
        sentenceID = row['SentenceID']
        doc = nlp(get_sentence(sentenceID))
    for ent in doc.ents:
        if row['Word'] == str(ent):
            predicted_df.loc[index,'Prediction'] = ent.label_
            break
    if str(predicted_df.loc[index,'Prediction']) == 'nan':
        predicted_df.loc[index,'Prediction'] = 'O'
    if index % 1000 == 0:
        print(str(index) + ' rows predicted')

predicted_df.to_csv('../datasets/TrainingPrediction2.csv', sep=';')
predicted_df.tail()

0 rows predicted
1000 rows predicted
2000 rows predicted
3000 rows predicted
4000 rows predicted
5000 rows predicted
6000 rows predicted
7000 rows predicted
8000 rows predicted
9000 rows predicted
10000 rows predicted
11000 rows predicted
12000 rows predicted
13000 rows predicted
14000 rows predicted
15000 rows predicted
16000 rows predicted
17000 rows predicted
18000 rows predicted
19000 rows predicted
20000 rows predicted
21000 rows predicted
22000 rows predicted
23000 rows predicted
24000 rows predicted
25000 rows predicted
26000 rows predicted
27000 rows predicted
28000 rows predicted
29000 rows predicted
30000 rows predicted
31000 rows predicted
32000 rows predicted
33000 rows predicted
34000 rows predicted
35000 rows predicted
36000 rows predicted
37000 rows predicted
38000 rows predicted
39000 rows predicted
40000 rows predicted
41000 rows predicted
42000 rows predicted
43000 rows predicted
44000 rows predicted
45000 rows predicted
46000 rows predicted
47000 rows predicted
48000

,Sentence #,Word,POS,Tag,SentenceID,Prediction
196641,NaN,and,CC,O,9000.0,O
196642,NaN,foreign,JJ,O,9000.0,O
196643,NaN,investment,NN,O,9000.0,O
196644,NaN,.,.,O,9000.0,O
196645,9001,NaN,NaN,NaN,9000.0,O


In [233]:
print("Acc: " + str(get_acc(predicted_df)))
print("Recall: " + str(get_recall(predicted_df)))
print("Precision: " + str(get_precision(predicted_df)))

Acc: 91.86456881909625
Recall: 54.61113330669863
Precision: 0.5595238095238095


In [63]:
# errors = 0
# for index, row in validation.iterrows():
#     doc = nlp(str(row['Word']))
#     validation.loc[index, 'Prediction'] = doc.ents[0].label_
#     if doc.ents[0].label_ != str(row['Tag']):
#         errors += 1
# print('Accuracy: ' + str(1 - errors/len(validation.index)))

Accuracy: 0.8798094037000499


In [64]:
validation.head()

,Sentence #,Word,POS,Tag,Prediction
0,Sentence: 1,President,NNP,B-per,B-per
1,NaN,Karzai,NNP,I-per,I-per
2,NaN,thanked,VBD,O,O
3,NaN,his,PRP$,O,O
4,NaN,allies,NNS,O,O


In [65]:
validation.to_csv('../datasets/TrainingPrediction.csv')

In [66]:
results = {
    "Size": validation.shape[0],
    "CorrectPredictionsWithO": validation.loc[validation['Tag'] == validation['Prediction']].shape[0],
    "CorrectEntity": validation.loc[(validation['Tag'] != "O") & (validation['Tag'] == validation['Prediction'])].shape[0],
    "AllEntities": validation.loc[validation['Tag'] != "O"].shape[0],
    "AllPredictions": validation.loc[validation['Prediction'] != "O"].shape[0]
}
results

{'AllEntities': 30036,
 'AllPredictions': 30266,
 'CorrectEntity': 10563,
 'CorrectPredictionsWithO': 173010,
 'Size': 196646}

In [67]:
print("Absoulte accuracy: {0:.2f}%".format(results["CorrectPredictionsWithO"] / results["Size"] * 100))
print("Entity based accuracy: {0:.2f}%".format(results["CorrectEntity"] / results["AllEntities"] * 100))
print("Predicted entity based accuracy: {0:.2f}%".format(results["CorrectEntity"] / results["AllPredictions"] * 100))

Absoulte accuracy: 87.98%
Entity based accuracy: 35.17%
Predicted entity based accuracy: 34.90%
